In [1]:
import setup
setup.init_django()

In [2]:
from blog.models import BlogPost


In [3]:
from openai import OpenAI
from decouple import config
EMBEDDING_MODEL= SentenceTransformer("all-MiniLM-L6-v2")
RECREATE_DATA=True

In [4]:
docs=["The dog jumped over the cat",
     'The cat jumped over the dog',
     'The weather is very hot',
    'the cat is yellow and dog is red']

In [5]:
new_data=[]
for i,x in enumerate(docs):
    new_data.append(BlogPost(title=f"Blog Post {i+1}", content=x, can_delete=True))
if RECREATE_DATA:    
    qs=BlogPost.objects.filter(can_delete=True) 
    qs.delete()
    qs=BlogPost.objects.bulk_create(new_data)

In [6]:
qs=BlogPost.objects.filter(can_delete=True)
qs.count()

4

In [7]:

client=OpenAI(
    api_key=config("OPENAI_API_KEY")
)

In [8]:
def get_embedding(text,model=EMBEDDING_MODEL):
    text=text.replace("\n","").strip()
    return EMBEDDING_MODEL.encode(text)[0][1]

In [9]:
EMBEDDING_MODEL

'text-embedding-3-small'

In [10]:
for obj in qs:
    if obj.embedding is None:
        obj.embedding=get_embedding(obj.get_embedding_text_raw())
        obj.save()
    

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}